In [1]:
import torch
if torch.cuda.is_available():
    torch.cuda.init()
    assert torch.cuda.is_initialized()
    print('Cuda is ready')    
    
import os
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from SAITS_functions import *

/DeepenData/.miniconda/envs/torch_aa/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cuda is ready


In [2]:
dataset_path =  f"{os.getcwd()}/data"

file_path = os.path.join(dataset_path, 'datasets.h5')
with h5py.File(file_path, 'r') as hf:  # read data from h5 file
    #X = hf['train']['X'][:]
    X0 = hf['train']['X'][:]
print(
X0.ndim,
X0.shape,
X0.size, dataset_path)

3 (15, 9, 51) 6885 /DeepenData/Repos/Tyrosinemia/deprecated/data


In [3]:

from functions_aa import *
args, model_args       = get_args(seq_len = 9, feature_num  = 51, batch_size   = 3)
assert args.dataset_path == dataset_path
masked_imputation_task = True
model_type             = 'SAITS'
args.batch_size        = 3
args.device            = 'cuda'
unified_dataloader     = UnifiedDataLoader(args.dataset_path, args.seq_len, args.feature_num, model_type, args.batch_size, args.num_workers, masked_imputation_task)
train_data, val_data, test_data = unified_dataloader.prepare_all_data_for_imputation()

Cuda is ready


In [4]:
import pickle

with open('sets_ids_splitted.pickle', 'rb') as handle:
    sets_ids_splitted = pickle.load(handle)    

sets_ids_splitted['train_set_ids']

idx_to_code_dict = dict(zip(range(15), sets_ids_splitted['train_set_ids']))
idx_to_code_dict

{0: 'T13',
 1: 'T14',
 2: 'T04',
 3: 'T18',
 4: 'T12',
 5: 'T08',
 6: 'T05',
 7: 'T07',
 8: 'T01',
 9: 'T06',
 10: 'T02',
 11: 'T10',
 12: 'T09',
 13: 'T03',
 14: 'T11'}

In [5]:
list(train_data).__len__(), list(train_data)[0].__len__(), list(train_data)[0].__len__(), list(train_data)[0][0].shape, list(train_data)[1][1].shape

idxs = []
for i, t, x in train_data:
    idxs.append(i)
    
a_over_sample = torch.cat(idxs).numpy().tolist()

import pickle

def idxs_to_codes(a_over_sample):

    with open('sets_ids_splitted.pickle', 'rb') as handle:
        sets_ids_splitted = pickle.load(handle)    


    idx_to_code_dict = dict(zip(range(15), sets_ids_splitted['train_set_ids']))


    return [idx_to_code_dict[s] for s in a_over_sample]

np.array(idxs_to_codes(a_over_sample))

array(['T08', 'T08', 'T01', 'T13', 'T14', 'T08', 'T13', 'T10', 'T11',
       'T02', 'T05', 'T14', 'T13', 'T02', 'T07', 'T13', 'T09', 'T10',
       'T06', 'T11', 'T10', 'T03', 'T06', 'T02', 'T11', 'T02', 'T08',
       'T11', 'T13', 'T13', 'T13', 'T05', 'T18'], dtype='<U3')

In [6]:
saved_parameters_path = f"{os.getcwd()}/state_dict_best_trained_model.pth"
model = SAITS(**model_args)
model.load_state_dict(torch.load(saved_parameters_path));
model.to('cuda');

model_args

{'device': 'cuda',
 'MIT': True,
 'n_groups': 5,
 'n_group_inner_layers': 1,
 'd_time': 9,
 'd_feature': 51,
 'dropout': 0.0,
 'd_model': 256,
 'd_inner': 512,
 'n_head': 8,
 'd_k': 32,
 'd_v': 32,
 'input_with_mask': True,
 'diagonal_attention_mask': True,
 'param_sharing_strategy': 'inner_group'}

In [7]:
model.to('cuda')
idxs = []
imputations = []


model.eval()
with torch.no_grad():
    for _, data  in enumerate(train_data):
        
        indices, X, missing_mask = map(lambda x: x.to(args.device), data)
        inputs = {'indices': indices, 'X': X, 'missing_mask': missing_mask}
        imputed_data, _ = model.impute(inputs)
        idxs.append(indices)
        imputations.append(imputed_data)
        
import pandas as pd
df_concat      = None
lista_tensores = []
codes          = []

df_concat = None
df        = None
for iteration, d in enumerate(zip(torch.cat(imputations), torch.cat(idxs))):
   
    t, i =  d    
    i    = i.cpu().detach().numpy().tolist()
    df   = pd.DataFrame(t.cpu().detach().numpy())
    df['codigo'] = i
    
    if iteration == 1 :       
        #print(9)
        df_concat = pd.concat([df_concat, df], axis=0)
        codes.append(i)
        
    else:
        if i not in codes:
            df_concat = pd.concat([df_concat, df], axis=0)
            codes.append(i)
            
    assert (df_concat.to_numpy() == df_concat.drop_duplicates().to_numpy()).all()      
df_concat

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,codigo
0,1.089000,9.950000,1.694225,18.299999,972.599976,75.900002,12.810000,1.000000,26.799999,0.800000,...,0.140000,0.060000,0.030000,0.060000,0.040000,0.080000,0.650000,0.420000,0.040000,5
1,0.557000,9.990000,0.495200,26.900000,1160.199951,67.099998,17.290001,1.090000,12.500000,0.386532,...,0.140000,0.070000,0.030000,0.080000,0.060000,0.100000,0.660000,0.410000,0.060000,5
2,1.079000,15.960000,2.205400,21.000000,1040.699951,63.500000,16.389999,1.190000,13.500000,-0.011307,...,0.150000,0.090000,0.030000,0.060000,0.030000,0.050000,0.730000,0.460000,0.020000,5
3,-0.006829,11.070034,0.528949,17.187136,346.499084,33.361362,15.658186,2.953495,13.728601,1.206563,...,-0.983593,-0.151089,0.866048,-0.157528,-0.864991,-2.717116,-0.037949,1.299838,0.576294,5
4,-0.043081,10.839651,0.614993,17.062031,352.360657,34.145393,15.599005,3.047254,13.710817,1.162856,...,-0.965096,-0.028388,0.816545,-0.099525,-0.920466,-2.634155,-0.025801,1.265713,0.543632,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,0.620751,0.000000,3.390000,22.299999,97.800003,40.599998,2.410000,1.200000,16.000000,0.310000,...,0.060000,0.010000,0.010000,0.010000,0.010000,0.040000,0.890000,0.340000,0.030000,0
5,-0.147142,6.579111,7.080000,11.900000,42.500000,39.099998,1.086957,1.060000,14.100000,0.350000,...,-0.736101,-3.850513,1.614697,0.572195,1.668759,-2.746896,-3.043094,1.777694,-0.341892,0
6,1.565061,0.000000,2.040000,18.459999,40.689999,35.889999,1.133742,1.020000,13.400000,0.370000,...,0.110000,0.010000,0.030000,0.040000,0.030000,0.050000,0.700000,0.290000,0.030000,0
7,1.391949,8.863811,1.920000,18.868975,45.000000,33.099998,1.359517,3.235786,13.400000,0.490000,...,0.100000,0.010000,0.010000,0.100000,0.040000,0.060000,0.740000,0.290000,0.040000,0


In [8]:
imputed_train_df = df_concat.copy()  

imputed_train_df['codigo'] = idxs_to_codes(df_concat['codigo'])
imputed_train_df

imputed_train_df.to_csv('imputed_train_df.csv')

In [9]:
model.to('cuda')
idxs = []
imputations = []


model.eval()
with torch.no_grad():
    for _, data  in enumerate(val_data):
        
        indices, X, missing_mask = map(lambda x: x.to(args.device), data)
        inputs = {'indices': indices, 'X': X, 'missing_mask': missing_mask}
        imputed_data, _ = model.impute(inputs)
        idxs.append(indices)
        imputations.append(imputed_data)
        
import pandas as pd
df_concat      = None
lista_tensores = []
codes          = []

df_concat = None
df        = None
for iteration, d in enumerate(zip(torch.cat(imputations), torch.cat(idxs))):
   
    t, i =  d    
    i    = i.cpu().detach().numpy().tolist()
    df   = pd.DataFrame(t.cpu().detach().numpy())
    df['codigo'] = i
    
    if iteration == 1 :       
        #print(9)
        df_concat = pd.concat([df_concat, df], axis=0)
        codes.append(i)
        
    else:
        if i not in codes:
            df_concat = pd.concat([df_concat, df], axis=0)
            codes.append(i)
            
    assert (df_concat.to_numpy() == df_concat.drop_duplicates().to_numpy()).all()      
df_concat

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,codigo
0,0.859000,63.029999,-0.395414,27.600000,1264.199951,30.100000,42.000000,3.357273,16.714983,-0.495561,...,0.190000,0.060000,0.030000,0.070000,0.050000,0.120000,0.710000,0.350000,0.030000,3
1,0.858000,61.119999,-0.875369,30.000000,1175.400024,101.300003,11.600000,2.517048,17.053160,-0.806570,...,0.090000,0.040000,0.010000,0.060000,0.020000,0.060000,0.610000,0.400000,0.020000,3
2,0.840000,56.259998,0.142400,34.599998,1105.099976,95.300003,11.600000,3.085670,16.619507,-1.066220,...,0.100000,0.070000,0.020000,0.030000,0.020000,0.040000,0.520000,0.360000,0.020000,3
3,0.078660,15.616392,-0.302086,19.143372,315.490143,33.944637,14.531370,2.956966,13.419987,1.136155,...,-0.994467,-1.378257,0.772215,0.391759,-0.437974,-2.635273,-1.699326,1.640554,1.177421,3
4,0.738000,9.599522,0.569822,14.705239,285.809479,34.887394,8.534142,1.100000,13.600000,0.820000,...,-1.385881,-3.239480,0.572446,1.587759,-0.821282,-2.898219,-1.493971,0.811255,0.152226,3
5,0.630000,15.543032,-0.287886,18.783070,316.524902,33.825066,14.430539,3.112494,13.496812,1.050528,...,-0.926430,-1.142139,0.670317,0.373791,-0.644070,-2.597200,-1.728955,1.583949,1.214081,3
6,0.640000,20.340000,0.080952,15.250000,268.829987,58.130001,4.624634,2.924384,11.500000,10.100000,...,0.170000,0.070000,0.010000,0.080000,0.050000,0.080000,0.890000,0.440000,0.080000,3
7,0.630000,16.329945,-1.246433,18.365662,291.862427,29.523361,14.564474,3.089090,12.883327,1.314452,...,-1.190867,-1.256563,0.789983,0.302100,-0.560763,-2.725155,-1.447094,2.037074,1.295008,3
8,-0.338958,17.807608,-1.035482,20.123528,472.050476,44.581615,13.308641,2.365553,13.125978,1.480749,...,0.090000,0.010000,0.010000,0.040000,0.030000,0.040000,0.490000,0.290000,0.020000,3
0,1.020000,0.560000,2.970000,27.200001,721.200012,56.299999,12.810000,3.652224,14.829279,1.780000,...,0.180000,0.090000,0.030000,0.080000,0.060000,0.140000,0.730000,0.450000,0.040000,1


In [10]:
idx_to_code_dict_val_set_ids = dict(zip(range(15), sets_ids_splitted['val_set_ids']))
idx_to_code_dict_val_set_ids

{0: 'T19', 1: 'T17', 2: 'T16', 3: 'T15'}

In [11]:
def idxs_to_codes_val_set_ids(a_over_sample):

    with open('sets_ids_splitted.pickle', 'rb') as handle:
        sets_ids_splitted = pickle.load(handle)    


    idx_to_code_dict = dict(zip(range(15), sets_ids_splitted['val_set_ids']))


    return [idx_to_code_dict[s] for s in a_over_sample]


imputed_val_df = df_concat.copy()  

imputed_val_df['codigo'] = idxs_to_codes_val_set_ids(df_concat['codigo'])
imputed_val_df

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,codigo
0,0.859000,63.029999,-0.395414,27.600000,1264.199951,30.100000,42.000000,3.357273,16.714983,-0.495561,...,0.190000,0.060000,0.030000,0.070000,0.050000,0.120000,0.710000,0.350000,0.030000,T15
1,0.858000,61.119999,-0.875369,30.000000,1175.400024,101.300003,11.600000,2.517048,17.053160,-0.806570,...,0.090000,0.040000,0.010000,0.060000,0.020000,0.060000,0.610000,0.400000,0.020000,T15
2,0.840000,56.259998,0.142400,34.599998,1105.099976,95.300003,11.600000,3.085670,16.619507,-1.066220,...,0.100000,0.070000,0.020000,0.030000,0.020000,0.040000,0.520000,0.360000,0.020000,T15
3,0.078660,15.616392,-0.302086,19.143372,315.490143,33.944637,14.531370,2.956966,13.419987,1.136155,...,-0.994467,-1.378257,0.772215,0.391759,-0.437974,-2.635273,-1.699326,1.640554,1.177421,T15
4,0.738000,9.599522,0.569822,14.705239,285.809479,34.887394,8.534142,1.100000,13.600000,0.820000,...,-1.385881,-3.239480,0.572446,1.587759,-0.821282,-2.898219,-1.493971,0.811255,0.152226,T15
5,0.630000,15.543032,-0.287886,18.783070,316.524902,33.825066,14.430539,3.112494,13.496812,1.050528,...,-0.926430,-1.142139,0.670317,0.373791,-0.644070,-2.597200,-1.728955,1.583949,1.214081,T15
6,0.640000,20.340000,0.080952,15.250000,268.829987,58.130001,4.624634,2.924384,11.500000,10.100000,...,0.170000,0.070000,0.010000,0.080000,0.050000,0.080000,0.890000,0.440000,0.080000,T15
7,0.630000,16.329945,-1.246433,18.365662,291.862427,29.523361,14.564474,3.089090,12.883327,1.314452,...,-1.190867,-1.256563,0.789983,0.302100,-0.560763,-2.725155,-1.447094,2.037074,1.295008,T15
8,-0.338958,17.807608,-1.035482,20.123528,472.050476,44.581615,13.308641,2.365553,13.125978,1.480749,...,0.090000,0.010000,0.010000,0.040000,0.030000,0.040000,0.490000,0.290000,0.020000,T15
0,1.020000,0.560000,2.970000,27.200001,721.200012,56.299999,12.810000,3.652224,14.829279,1.780000,...,0.180000,0.090000,0.030000,0.080000,0.060000,0.140000,0.730000,0.450000,0.040000,T17


In [12]:
imputed_val_df.to_csv('imputed_val_df.csv')